<a href="https://colab.research.google.com/github/purnimapatel/sweep_algo-on-sphere-in-python/blob/master/Voronoi__tessellation_on_sphere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#generating random points on a sphere wirh r=1,centre=0
import numpy as np
import math as m
def sample_sphere(npoints,ndim=3):
  vec=np.random.randn(ndim,npoints)
  vec/=np.linalg.norm(vec,axis=0)
  return(vec)
pts=sample_sphere(20).transpose() 
print(pts)

[[ 0.68337217 -0.47666354 -0.55298676]
 [-0.07638899  0.35333245 -0.9323738 ]
 [-0.47821292  0.87809536 -0.01615369]
 [ 0.48693098 -0.20631929  0.84872291]
 [-0.19520184 -0.82550224 -0.52956803]
 [ 0.83666117 -0.4488552  -0.31389026]
 [-0.86977593 -0.40919249  0.27577406]
 [ 0.83062405 -0.1736737  -0.52905684]
 [-0.98078305 -0.16363519  0.10624563]
 [ 0.04342446 -0.94823117  0.31459809]
 [-0.53659293  0.69900039  0.47272243]
 [ 0.90079511  0.33352233 -0.27808457]
 [ 0.75914052  0.45202131 -0.46838275]
 [ 0.95998063 -0.14886527 -0.23722632]
 [ 0.45718829 -0.88936166  0.00383542]
 [-0.06280483 -0.99452687  0.08349761]
 [-0.6992842   0.48106945 -0.52874739]
 [-0.02851699  0.74137414  0.67048577]
 [ 0.26334772  0.48264214  0.8352871 ]
 [ 0.0559372  -0.95968649 -0.2754503 ]]


In [6]:
#convertion of cartesian co-ordinate to spherical co-ordinate
def cart2sph(x,y,z):
  x2y2=x**2+y**2
  r=m.sqrt(x2y2+z**2)
  elev=m.atan2(z,m.sqrt(x2y2))
  az=m.atan2(y,x)
  return r,elev,az
def cart2spA(pts):
  return np.array([cart2sph(x,y,z) for x,y,z in pts])
print(cart2spA(pts))    

[[ 1.         -0.58594472 -0.60905741]
 [ 1.         -1.20092481  1.78371517]
 [ 1.         -0.01615439  2.06948614]
 [ 1.          1.01356569 -0.40078057]
 [ 1.         -0.55809125 -1.80299553]
 [ 1.         -0.31928762 -0.49240693]
 [ 1.          0.27939489 -2.70185722]
 [ 1.         -0.55748873 -0.20611876]
 [ 1.          0.10644654 -2.976274  ]
 [ 1.          0.32003322 -1.52503307]
 [ 1.          0.49237764  2.22550282]
 [ 1.         -0.28179945  0.35460266]
 [ 1.         -0.48745944  0.53705853]
 [ 1.         -0.23950967 -0.15384575]
 [ 1.          0.00383543 -1.09596139]
 [ 1.          0.08359493 -1.63386304]
 [ 1.         -0.5571241   2.53900283]
 [ 1.          0.73486334  1.60924242]
 [ 1.          0.98865476  1.0713085 ]
 [ 1.         -0.27905809 -1.51257525]]


In [ ]:
#Visualization of the points.
from matplotlib import pyplot as plt
from